In [ ]:
from MySQL import MySQL_Run

MySQL_Run("show tables")

In [33]:
MySQL_Run("""DELETE FROM reservation
WHERE user_id = '124';
""")

'Query executed.'

In [ ]:
Results = MySQL_Run(f"select * from users")
df = pd.DataFrame(Results)
df

In [ ]:
import pandas as pd

Res = "user_id, booking_time, booking_number, booking_start_station_name, booking_end_station_name"
Res2 = "user_id, booking_time, booking_number, booking_start_station_name, booking_end_station_name, review_status, payment_status"
Val = "1, '2002-06-20 10:00:00', '150', '工109', '工515'"

def Show_Resarvation(Res2):
    Results = MySQL_Run(f"select {Res2} from reservation")
    df = pd.DataFrame(Results)
    return df

def Push_Resarvation(Val):
    sql = f"insert into reservation ({Res}) values ({Val})"
    MySQL_Run(sql)
    return "Success"

# Push_Resarvation(Val)
Show_Resarvation(Res2)

In [35]:
sql = """
SELECT user_id, booking_time, booking_number, 
        booking_start_station_name, booking_end_station_name,
        review_status, payment_status
FROM reservation where user_id = '124' order by created_at desc
"""
results = MySQL_Run(sql)
results

[{'user_id': 124,
  'booking_time': datetime.datetime(2025, 9, 15, 19, 23),
  'booking_number': 1,
  'booking_start_station_name': '將軍府',
  'booking_end_station_name': '花蓮港景觀橋',
  'review_status': 'pending',
  'payment_status': 'pending'}]